# Load topographic map data from subjects

In [ ]:
#-------------------------------------------------------------------------------
FILEID = "12Atu_t9xpPChdd1hhKNv2RCiyAVdyUud"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O GIGAnew_data_all_sbjs.zip && rm -rf /tmp/cookies.txt
!unzip GIGAnew_data_all_sbjs.zip
!dir
!pip install mne==0.19
#-------------------------------------------------------------------------------

# Load pre-trained weights (Source-single)

In [ ]:
#-------------------------------------------------------------------------------
FILEID = "1e8NrT4jCSGC_9LWYH0zkfAa1CxkIpD0J"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O GIGAnew_weights_all_sbjs3.zip && rm -rf /tmp/cookies.txt
!unzip GIGAnew_weights_all_sbjs3.zip
!dir
!pip install mne==0.19
#-------------------------------------------------------------------------------

# Loading supporting modules

In [ ]:
#-------------------------------------------------------------------------------
import numpy as np
import scipy.io as sio
import pywt
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import math
import shutil
import pickle
import os
import random
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix,cohen_kappa_score,mean_squared_error
from tensorflow import keras
from tensorflow.keras.constraints import max_norm
from datetime import date, datetime
from scipy.stats import kurtosis, skew
from sklearn.model_selection import ShuffleSplit
from time import time
#-------------------------------------------------------------------------------

# Define custom functions to use
- We design three custom functions: i) load data acroos time windows, ii) normalization function, and iii) function to build the CNN network.

In [ ]:
#-------------------------------------------------------------------------------
def TW_data(sbj,time_inf,time_sup):
    # Load data/images----------------------------------------------------------
    path_cwt = 'CWT_CSP_data_mubeta_8_30_Tw_'+str(time_inf)+'s_'+str(time_sup)+'s_subject'+str(sbj)+'_cwt_resized_10.pickle'  
    with open(path_cwt, 'rb') as f:
         X_train_re_cwt, X_test_re_cwt, y_train, y_test = pickle.load(f)
    path_csp = 'CWT_CSP_data_mubeta_8_30_Tw_'+str(time_inf)+'s_'+str(time_sup)+'s_subject'+str(sbj)+'_csp_resized_10.pickle'  
    with open(path_csp, 'rb') as f:
         X_train_re_csp, X_test_re_csp, y_train, y_test = pickle.load(f)
    #---------------------------------------------------------------------------
    return X_train_re_cwt, X_train_re_csp, X_test_re_cwt, X_test_re_csp, y_train, y_test
#-------------------------------------------------------------------------------
def norm_data(XF_train_cwt, XF_train_csp, XF_test_cwt, XF_test_csp, n_fb, Ntw, y_train, y_test):
    # orden de las inputs:
    # [CWT_fb1_TW1, CWT_fb2_TW1 --- CWT_fb1_TW2, CWT_fb2_TW2 --- CWT_fb1_TWN, CWT_fb2_TWN] ... [CSP]
    #---------------------------------------------------------------------------
    XT_train_csp = []
    XT_valid_csp = []
    XT_test_csp  = []
    XT_train_cwt = []
    XT_valid_cwt = []
    XT_test_cwt  = []
    for tw in range(Ntw):
        for fb in range(n_fb):
            X_train_cwt, X_test_cwt = XF_train_cwt[tw][:,fb,:,:].astype(np.uint8), XF_test_cwt[tw][:,fb,:,:].astype(np.uint8)
            X_train_csp, X_test_csp = XF_train_csp[tw][:,fb,:,:].astype(np.uint8), XF_test_csp[tw][:,fb,:,:].astype(np.uint8)
            #-------------------------------------------------------------------
            # train/validation data split
            rs = ShuffleSplit(n_splits=1, test_size=.1, random_state=0)
            for train_index, valid_index in rs.split(X_train_cwt):
              X_train_cwtf = X_train_cwt[train_index,:,:] # cwt
              X_valid_cwtf = X_train_cwt[valid_index,:,:]
              X_train_cspf = X_train_csp[train_index,:,:] # csp
              X_valid_cspf = X_train_csp[valid_index,:,:]
            #-------------------------------------------------------------------          
            # Normalize data----------------------------------------------------
            X_mean_cwt  = X_train_cwtf.mean(axis=0, keepdims=True)
            X_std_cwt   = X_train_cwtf.std(axis=0, keepdims=True) + 1e-7
            X_train_cwt = (X_train_cwtf - X_mean_cwt) / X_std_cwt
            X_valid_cwt = (X_valid_cwtf - X_mean_cwt) / X_std_cwt
            X_test_cwt  = (X_test_cwt  - X_mean_cwt) / X_std_cwt
                                
            X_mean_csp  = X_train_cspf.mean(axis=0, keepdims=True)
            X_std_csp   = X_train_cspf.std(axis=0, keepdims=True) + 1e-7
            X_train_csp = (X_train_cspf - X_mean_csp) / X_std_csp
            X_valid_csp = (X_valid_cspf - X_mean_csp) / X_std_csp
            X_test_csp  = (X_test_csp  - X_mean_csp) / X_std_csp
            #-------------------------------------------------------------------
            # set new axis------------------------------------------------------
            X_train_cwt = X_train_cwt[..., np.newaxis]
            X_valid_cwt = X_valid_cwt[..., np.newaxis]
            X_test_cwt  = X_test_cwt[..., np.newaxis]   
            XT_train_cwt.append(X_train_cwt)
            XT_valid_cwt.append(X_valid_cwt)
            XT_test_cwt.append(X_test_cwt)
                                
            X_train_csp = X_train_csp[..., np.newaxis]
            X_valid_csp = X_valid_csp[..., np.newaxis]
            X_test_csp  = X_test_csp[..., np.newaxis]   
            XT_train_csp.append(X_train_csp)
            XT_valid_csp.append(X_valid_csp)
            XT_test_csp.append(X_test_csp)
            #-------------------------------------------------------------------
    y_trainf = y_train[train_index]
    y_validf = y_train[valid_index]
    y_trainF, y_validF, y_testF = y_trainf.reshape((-1,))-1, y_validf.reshape((-1,))-1, y_test.reshape((-1,))-1
    #---------------------------------------------------------------------------
    # Convert class vectors to binary class matrices----------------------------
    y_train = keras.utils.to_categorical(y_trainF,num_classes)
    y_valid = keras.utils.to_categorical(y_validF,num_classes)
    y_test  = keras.utils.to_categorical(y_testF,num_classes)
    #---------------------------------------------------------------------------
    XT_train = XT_train_cwt + XT_train_csp
    XT_valid = XT_valid_cwt + XT_valid_csp
    XT_test  = XT_test_cwt  + XT_test_csp
    #---------------------------------------------------------------------------
    return XT_train, XT_valid, XT_test, y_train, y_valid, y_test 
#-------------------------------------------------------------------------------
def cnn_network(n_fb,Nkfeats,Ntw,shape_,n_filt,units,l1p,l2p,lrate,sbj):
    #---------------------------------------------------------------------------
    keras.backend.clear_session()
    np.random.seed(123)
    tf.compat.v1.random.set_random_seed(123)
    #---------------------------------------------------------------------------
    input_  = [None]*Ntw*n_fb*Nkfeats
    conv_   = [None]*Ntw*n_fb*Nkfeats
    pool_   = [None]*Ntw*n_fb*Nkfeats
    batch0_ = [None]*Ntw*n_fb*Nkfeats
    for i in range(Ntw*n_fb*Nkfeats):
        input_[i]  = keras.layers.Input(shape=[shape_,shape_,1])
        conv_[i]   = keras.layers.Conv2D(filters=n_filt,kernel_size=3,strides=1,activation='relu',padding='SAME',input_shape=[shape_,shape_,1])(input_[i])
        #-----------------------------------------------------------------------
        batch0_[i] = keras.layers.BatchNormalization()(conv_[i])
        #-----------------------------------------------------------------------
        pool_[i]   = keras.layers.MaxPooling2D(pool_size=2)(batch0_[i])
    concat  = keras.layers.concatenate(pool_)
    flat    = keras.layers.Flatten()(concat)
    batch1  = keras.layers.BatchNormalization()(flat)
    hidden1 = keras.layers.Dense(units=units,activation='relu',kernel_regularizer=keras.regularizers.l1_l2(l1=l1p, l2=l2p), kernel_constraint=max_norm(1.))(batch1)#
    batch2  = keras.layers.BatchNormalization()(hidden1)
    output  = keras.layers.Dense(units=2, activation='softmax', kernel_constraint=max_norm(1.))(batch2)#
    model   = keras.models.Model(inputs=input_, outputs=[output])
    
    learning_rate_fn = tf.keras.optimizers.schedules.PolynomialDecay(lrate, 100, power=1.0,cycle=False, name=None)
    opt     = keras.optimizers.Adam(learning_rate=learning_rate_fn) 
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
    return model
#-------------------------------------------------------------------------------

# Creating results and parameter setting folders
- In folder results we store the performance values in format .mat
- In folder parameter_setting we store the model' weights in format .hdf5 per fold.

In [ ]:
#-------------------------------------------------------------------------------
try:
  os.mkdir('results')
except:
  print("Exist folder!")
#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------
try:
  os.mkdir('parameter_setting')
except:
  print("Exist folder!")
#-------------------------------------------------------------------------------

# DWCNN training, validation and test stage using pre-trained weights from source subject to initilize the model
In this example, we consider one fold training and validation.

In [ ]:
# Some initial parameters-------------------------------------------------------
sbj           = 40                             # subject id (target)
tr_sbj        = 36                             # subject id (source)
n_fb          = 2                              # 0 --> 8-12Hz, 1--->12-30Hz
num_classes   = 2                              # num of classes
n_filt        = 2                              # number of CNN filters
learning_rate = [1e-3]                         # learning rate value
l1_param      = [0.0005,0.001,0.005]           # l_1 regularization param
l2_param      = [0.0005,0.001,0.005]           # L_2 regularization param
Ntw           = 5                              # number of time windows
acc_test      = []                             # store variable of acc validation
Nkfeats       = 2                              # number of feat types
#-------------------------------------------------------------------------------
# load data train/test trough all tw
th_name  = np.array([[-1.5, 0.5],[-0.5, 1.5],[0.5, 2.5],[1.5, 3.5],[2.5, 4.5]]) # considered time-windows
XF_train_cwt = []
XF_train_csp = []
XF_test_cwt  = []
XF_test_csp  = []
for i in range(th_name.shape[0]):
  X_train_re_cwt, X_train_re_csp, X_test_re_cwt, X_test_re_csp, y_trainF, y_testF = TW_data(sbj,th_name[i,0],th_name[i,1])
  XF_train_cwt.append(X_train_re_cwt)
  XF_train_csp.append(X_train_re_csp)
  XF_test_cwt.append(X_test_re_cwt)
  XF_test_csp.append(X_test_re_csp)
#-------------------------------------------------------------------------------
# adjust data
XT_train, XT_valid, XT_test, y_train, y_valid, y_test = norm_data(XF_train_cwt, XF_train_csp, XF_test_cwt, XF_test_csp, n_fb, Ntw, y_trainF, y_testF)
print('Train', XT_train[0].shape,y_train.shape)
print('Valid', XT_valid[0].shape,y_valid.shape)
print('Test', XT_test[0].shape,y_test.shape)
#-------------------------------------------------------------------------------
files_     = os.listdir("./")
files_sbj_ = [s for s in files_ if 'sbj_'+str(tr_sbj)+'_' in s]
print(files_sbj_)
id_sbj  = files_sbj_[0].find('units_')
id_l1   = files_sbj_[0].find('_l1_')
n_unit  = int(files_sbj_[0][id_sbj+6:id_l1])
#-------------------------------------------------------------------------------
start = time()
for lr_p in range(len(learning_rate)):# loop across lr param
  for l1_p in range(len(l1_param)):# loop across l1 param
      for l2_p in range(len(l2_param)):# loop across l2 param
          #---------------------------------------------------------------------
          # define cnn network
          model = cnn_network(n_fb,Nkfeats,Ntw,40,n_filt,n_unit,l1_param[l1_p],l2_param[l2_p],learning_rate[lr_p],sbj)
          #---------------------------------------------------------------------
          # load learned weights
          filepath0        = files_sbj_[0]
          checkpoint_path0 = filepath0
          model.load_weights(checkpoint_path0)
          #---------------------------------------------------------------------
          # checkpoint
          filepath   ='parameter_setting/weights_sbj_'+str(sbj)+'_filters_'+str(n_filt)+'_units_'+str(n_unit)+'_lr_'+str(learning_rate[lr_p])+'_l1_'+str(l1_param[l1_p])+'_l2_'+str(l2_param[l2_p])+'_source_'+str(tr_sbj)+'.hdf5'
          checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
          #---------------------------------------------------------------------
          # Fit the CNN model
          history = model.fit(XT_train, y_train, 
                                epochs=200, 
                                batch_size=256, 
                                validation_data=(XT_valid,y_valid),
                                callbacks=[checkpoint, keras.callbacks.EarlyStopping(patience=200,restore_best_weights=True)],verbose=2)
          #---------------------------------------------------------------------
          # Predict model and confusion matrix
          model.load_weights(filepath)
          y_test_pred = model.predict(XT_test)
          y_test_pred = np.argmax(y_test_pred, axis=1)
          y_test1     = np.argmax(y_test,axis=1)
          AccV        = 1-mean_squared_error(y_test1,y_test_pred)
          kappaV      = cohen_kappa_score(y_test1, y_test_pred, labels=None, weights=None)
          #---------------------------------------------------------------------      
          # plot loss performance    
          plt.plot(history.history['loss'])
          plt.plot(history.history['val_loss'])
          plt.title('model loss')
          plt.ylabel('loss')
          plt.xlabel('epoch')
          plt.legend(['train', 'valid'], loc='upper left')
          plt.show()
          #---------------------------------------------------------------------
          with open('parameter_setting/tr_vld_loss_sbj_'+str(sbj)+'_units_'+str(n_unit)+'_lr_'+str(learning_rate[lr_p])+'_l1_'+str(l1_param[l1_p])+'_l2_'+str(l2_param[l2_p])+'_source_'+str(tr_sbj)+'.pickle', 'wb') as f:
            pickle.dump([history.history['loss'], history.history['val_loss']], f)
          #---------------------------------------------------------------------
          del model
          #---------------------------------------------------------------------
          acc_test.append([sbj,
                          n_filt,
                          n_unit,
                          learning_rate[lr_p],
                          l1_param[l1_p],
                          l2_param[l2_p],
                          AccV*100, 
                          kappaV])
          sio.savemat('results/Sbj_'+str(sbj)+'.mat',{'Acc':acc_test})
          #---------------------------------------------------------------------
print('CNN train/validation Done!!!\n')
print('Time for CNN training and validation:', time()-start)
#-------------------------------------------------------------------------------

# Auxiliar definition

In [ ]:
# definitions
#-------------------------------------------------------------------------------
def min_dist(X):
    p1       = [100.0,1.0]
    distance = []
    for i in range(len(X)):
        p2 = X[i,:]
        distance.append(math.sqrt( ((p1[0]-p2[0])**2)+((p1[1]-p2[1])**2) ))
    min_id = np.argmin(distance) 
    return distance, min_id
#-------------------------------------------------------------------------------

# Cuantitative results (classification performance on test set)

In [ ]:
#-------------------------------------------------------------------------------
Nsbj        = 40
n_fb        = 2
num_classes = 2
print('Subject '+str(Nsbj))
# load rithms data--------------------------------------------------------------
data = sio.loadmat('results/Sbj_'+str(Nsbj)+'.mat')
data = data['Acc']
#print(data)
X    = data[:,data.shape[1]-2:data.shape[1]]
print(X)
#-------------------------------------------------------------------------------    
# select best performance-------------------------------------------------------
distance, min_id = min_dist(X)
max_id           = min_id
opt_neurons      = data[max_id,data.shape[1]-6]
opt_lr           = data[max_id,data.shape[1]-5]
opt_l1           = data[max_id,data.shape[1]-4]
opt_l2           = data[max_id,data.shape[1]-3]
print('Subject '+str(Nsbj)+ '- acc: ' +str(X[max_id,0])+ '- Kappa value: '+str(X[max_id,1])+
      ' neurons '+str(int(opt_neurons))+' lr ' +str(opt_lr)+' l1_val '+str(opt_l1)+ ' l2_val '+str(opt_l2))
#-------------------------------------------------------------------------------

# Qualitative results (Interpretability of results according relevant brain areas)
- Reconstructed spatial representations that highlights the most contributing information in all domains (Type of EEG representation, filter band and time window.)

In [ ]:
#-------------------------------------------------------------------------------
def hidden_layer_results(hidden_layer,shape_im):
    weights, biases = hidden_layer.get_weights()    
    tmp0            = np.sum(np.abs(weights),axis=1)
    n_filters       = np.int(tmp0.shape[0]/(shape_im[0]*shape_im[1]))
    im              = [] 
    for i in range(n_filters):
        image = []
        j     = 0 + i
        while j < tmp0.shape[0]:
            image.append(tmp0[j])
            j = j+(n_filters)
        image = np.array(image)
        im.append(image.reshape((shape_im)))
    im  = np.array(im)
    # average according frequency bands
    vec_t = np.arange(0,im.shape[0],2)
    imF   = []
    for l in range(len(vec_t)):
        if l == len(vec_t)-1:
            imF.append(np.mean(im[vec_t[l]:im.shape[0],:,:],axis=0))
        else:
            imF.append(np.mean(im[vec_t[l]:vec_t[l+1],:,:],axis=0))
    imF = np.array(imF)
    return imF
#-------------------------------------------------------------------------------

In [ ]:
#-------------------------------------------------------------------------------
names_x  = [r'-1.5s-0.5s',r'$-0.5s-1.5s$',r'$0.5s-2.5s$',r'$1.5s-3.5s$',r'$2.5s-4.5s$']
Ntw      = 5
n_fb     = 2
Nkfeats  = 2
hist     = []
print('Subject '+str(Nsbj))
# figure plot setting-----------------------------------------------------------
fig, axs = plt.subplots(4,5,figsize=(10,5.3))
fig.subplots_adjust(hspace = 0.1, wspace=.0001)
#-------------------------------------------------------------------------------
# Describe CNN model------------------------------------------------------------
model = cnn_network(n_fb,Nkfeats,Ntw,40,n_filt,opt_neurons,opt_l1,opt_l2,learning_rate,Nsbj)
#-------------------------------------------------------------------------------
# plot topographic maps---------------------------------------------------------
topo_avg_mu_cwt = []
topo_avg_be_cwt = []
topo_avg_mu_csp = []
topo_avg_be_csp = []
for tw in range(Ntw):
  if tw == 0:
    ids_tw = [tw, tw+1, tw+10, tw+10+1] 
  else:
    ids_tw = [tw*2, tw*2+1, tw*2+10, tw*2+10+1]
  print(ids_tw)
  XT_mu_cwt = []
  XT_be_cwt = []
  XT_mu_csp = []
  XT_be_csp = []
  #-----------------------------------------------------------------------------
  filepath        = 'parameter_setting/weights_sbj_'+str(Nsbj)+'_filters_2_units_'+str(int(opt_neurons))+'_lr_'+str(opt_lr)+'_l1_'+str(opt_l1)+'_l2_'+str(opt_l2)+'_source_'+str(tr_sbj)+'.hdf5'
  checkpoint_path = filepath
  model.load_weights(checkpoint_path)
  #-----------------------------------------------------------------------------  
  # Hidden layer results
  hidden_layer = model.layers[83]
  shape_im     = model.layers[79].output_shape[1:3]
  im_mean      = hidden_layer_results(hidden_layer,shape_im)
  #-----------------------------------------------------------------------------    
  # standarsize and normalize
  x_mu_cwt   = np.array(im_mean[ids_tw[0],:,:])
  x_beta_cwt = np.array(im_mean[ids_tw[1],:,:])
  x_mu_csp   = np.array(im_mean[ids_tw[2],:,:])
  x_beta_csp = np.array(im_mean[ids_tw[3],:,:])
  #-----------------------------------------------------------------------------
  topo_avg_mu_cwt.append(np.array(x_mu_cwt))
  topo_avg_be_cwt.append(np.array(x_beta_cwt))
  topo_avg_mu_csp.append(np.array(x_mu_csp))
  topo_avg_be_csp.append(np.array(x_beta_csp))
  #-----------------------------------------------------------------------------
topo_avg_muT_cwt = np.array(topo_avg_mu_cwt)
topo_avg_beT_cwt = np.array(topo_avg_be_cwt)
topo_avg_muT_csp = np.array(topo_avg_mu_csp)
topo_avg_beT_csp = np.array(topo_avg_be_csp)
#-------------------------------------------------------------------------------
max_val = max(topo_avg_muT_cwt.max(),topo_avg_beT_cwt.max(),topo_avg_muT_csp.max(),topo_avg_beT_csp.max())
print('max_val',max_val)
#-------------------------------------------------------------------------------
topo_avg_muT_cwt = topo_avg_muT_cwt / max_val 
topo_avg_beT_cwt = topo_avg_beT_cwt / max_val 
topo_avg_muT_csp = topo_avg_muT_csp / max_val 
topo_avg_beT_csp = topo_avg_beT_csp / max_val
#-------------------------------------------------------------------------------
max_val = max(topo_avg_muT_cwt.max(),topo_avg_beT_cwt.max(),topo_avg_muT_csp.max(),topo_avg_beT_csp.max())
print('new_max_val',max_val)
#-------------------------------------------------------------------------------
min_val = min(topo_avg_muT_cwt.min(),topo_avg_beT_cwt.min(),topo_avg_muT_csp.min(),topo_avg_beT_csp.min())
print('min_val',min_val)
#-------------------------------------------------------------------------------
for vnt in range(Ntw):
    axs[0,vnt].imshow(topo_avg_muT_cwt[vnt,:,:],vmin=min_val, vmax=max_val)
    axs[1,vnt].imshow(topo_avg_beT_cwt[vnt,:,:],vmin=min_val, vmax=max_val)
    axs[2,vnt].imshow(topo_avg_muT_csp[vnt,:,:],vmin=min_val, vmax=max_val)
    axs[3,vnt].imshow(topo_avg_beT_csp[vnt,:,:],vmin=min_val, vmax=max_val)
    axs[3,vnt].set(xlabel=names_x[vnt])
    axs[3,vnt].xaxis.get_label().set_fontsize(15)
    if vnt == 0:
       axs[0,vnt].set(ylabel=r'$CWT \mu$')
       axs[0,vnt].yaxis.get_label().set_fontsize(15)
       axs[1,vnt].set(ylabel=r'$CWT \beta$')
       axs[1,vnt].yaxis.get_label().set_fontsize(15)
       axs[2,vnt].set(ylabel=r'$CSP \mu$')
       axs[2,vnt].yaxis.get_label().set_fontsize(15)
       axs[3,vnt].set(ylabel=r'$CSP \beta$')
       axs[3,vnt].yaxis.get_label().set_fontsize(15)
    fig.suptitle('Subject '+str(Nsbj), fontsize=16)

    # hist
    hist.append(topo_avg_muT_cwt[vnt,:,:].flatten())
    hist.append(topo_avg_beT_cwt[vnt,:,:].flatten())
    hist.append(topo_avg_muT_csp[vnt,:,:].flatten())
    hist.append(topo_avg_beT_csp[vnt,:,:].flatten())
#-------------------------------------------------------------------------------
for ax in axs.flat:
    ax.label_outer()
for ax in axs.flat:
    ax.set_xticks([])
    ax.set_yticks([])
    plt.savefig('results/Topoplots_Subject_'+str(Nsbj)+'T_source_'+str(tr_sbj)+'.svg', format='svg')
#-------------------------------------------------------------------------------
#plot histograms
hist = np.concatenate(hist)
hist = hist.flatten()
plt.figure(figsize=(5,5))
plt.hist(hist,bins=80)
plt.grid()
#-------------------------------------------------------------------------------

**NOTE:**
- The files of optimal model, the performance file, and the figures can be downloaded from current session COLAB.

# Download data

In [ ]:
#from google.colab import files
#files.download('/content/parameter_setting/weights_sbj_'+str(sbj)+'_filters_'+str(n_filt)+'_units_'+str(int(opt_neurons))+'_lr_'+str(opt_lr)+'_l1_'+str(opt_l1)+'_l2_'+str(opt_l2)+'_source_'+str(tr_sbj)+'.hdf5') 

In [ ]:
#from google.colab import files
#files.download('/content/parameter_setting/tr_vld_loss_sbj_'+str(sbj)+'_units_'+str(int(opt_neurons))+'_lr_'+str(opt_lr)+'_l1_'+str(opt_l1)+'_l2_'+str(opt_l2)+'_source_'+str(tr_sbj)+'.pickle') 

In [ ]:
#from google.colab import files
#files.download('/content/results/Topoplots_Subject_'+str(Nsbj)+'T_source_'+str(tr_sbj)+'.svg')